# Доклеим к wallets_features данные о пользователе

In [89]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(style="darkgrid")

np.random.seed(0)
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [90]:
wallets_features_raw = pd.read_csv("../dataset/Elliptic++ Dataset/wallets_features_classes_combined.csv")
wallets_to_user = pd.read_csv("../dataset/custom/AddrUser.csv")
user_data = pd.read_csv("../dataset/custom/UserData.csv")

In [91]:
wallets_with_features = pd.merge(
    left=wallets_features_raw,
    right=wallets_to_user,
    left_on="address",
    right_on="addrId",
    how="left",
)
wallets_with_features.drop("address", axis=1, inplace=True)


In [92]:
user_data.columns

Index(['userId', 'addr_cnt', 'outcoming_tx_cnt', 'incoming_tx_cnt',
       'input_users_cnt', 'output_users_cnt', 'class', 'active_time_steps_cnt',
       'btc_transacted_total', 'btc_sent_total', 'btc_received_total',
       'btc_sent_median', 'btc_received_median',
       'interracted_output_address_cnt', 'interracted_input_address_cnt',
       'overall_activity_coef', 'user_ts_fees_share_mean',
       'user_ts_fees_share_min', 'user_ts_fees_share_max', 'whole_fee_4',
       'whole_fee_5', 'whole_fee_6'],
      dtype='object')

In [93]:
columns_to_leave = [
    'userId',
    'user_ts_fees_share_mean',
    'user_ts_fees_share_min', 'user_ts_fees_share_max',
    'addr_cnt', 'outcoming_tx_cnt', 'incoming_tx_cnt',
    'input_users_cnt', 'output_users_cnt', 'active_time_steps_cnt',
    'btc_sent_total', 'btc_received_total',
    'interracted_output_address_cnt', 'interracted_input_address_cnt',
    'overall_activity_coef', 
    'whole_fee_5',
]
users_data_prepared = user_data[columns_to_leave]
users_data_prepared = users_data_prepared \
    .rename(columns={
        **{
            key: f"user_{key}" for key in columns_to_leave[4:]
        }
    })
users_data_prepared.columns

Index(['userId', 'user_ts_fees_share_mean', 'user_ts_fees_share_min',
       'user_ts_fees_share_max', 'user_addr_cnt', 'user_outcoming_tx_cnt',
       'user_incoming_tx_cnt', 'user_input_users_cnt', 'user_output_users_cnt',
       'user_active_time_steps_cnt', 'user_btc_sent_total',
       'user_btc_received_total', 'user_interracted_output_address_cnt',
       'user_interracted_input_address_cnt', 'user_overall_activity_coef',
       'user_whole_fee_5'],
      dtype='object')

In [94]:
wallets_with_users_features = pd.merge(
    left=wallets_with_features,
    right=users_data_prepared,
    left_on="userId",
    right_on="userId",
    how="left",
)
wallets_with_users_features.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1268260 entries, 0 to 1268259
Data columns (total 74 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   Time step                            1268260 non-null  int64  
 1   class                                1268260 non-null  int64  
 2   num_txs_as_sender                    1268260 non-null  float64
 3   num_txs_as receiver                  1268260 non-null  float64
 4   first_block_appeared_in              1268260 non-null  float64
 5   last_block_appeared_in               1268260 non-null  float64
 6   lifetime_in_blocks                   1268260 non-null  float64
 7   total_txs                            1268260 non-null  float64
 8   first_sent_block                     1268260 non-null  float64
 9   first_received_block                 1268260 non-null  float64
 10  num_timesteps_appeared_in            1268260 non-null  float64
 11

# Добавим новые фичи по кошелькам

- Активность (gini, число timestep)
- Число целых fees
- Отношение total_fees к средним по timestep по пользователю 

## Метрики активности

число timestep - уже посчитано в num_timesteps_appeared_in

In [95]:
wallet_to_ts = wallets_with_users_features[["addrId", "Time step"]]
wallet_to_ts = wallet_to_ts.drop_duplicates(subset=['addrId', 'Time step'])

In [96]:
def gini_coefficient(values):
    sorted_values = sorted(values)
    n = len(values)
    cumsum = sum([(i+1) * val for i, val in enumerate(sorted_values)])
    return (2 * cumsum) / (n * sum(values)) - (n + 1) / n
def safe_gini_coefficient(values):
    if len(values) < 2:
        return 0
    if sum(values) == 0:
        return 0
    try:
        return gini_coefficient(values)
    except:
        return 0

wallet_to_gini = wallet_to_ts.groupby('addrId')['Time step'].apply(safe_gini_coefficient)

In [97]:
wallet_to_gini = wallet_to_gini.reset_index().rename(columns={"Time step": "addr_gini"})

## Число целых fees

In [102]:
wallets_to_tx = pd.read_csv("../dataset/Elliptic++ Dataset/AddrTx_edgelist.csv")
tx_to_fee = pd.read_csv("../dataset/Elliptic++ Dataset/txs_features.csv")[["txId", "fees", "Time step"]]

In [103]:
wallets_to_tx_features = pd.merge(
    left=wallets_to_tx,
    right=tx_to_fee,
    left_on="txId",
    right_on="txId",
    how="left",
)
wallets_to_tx_features = wallets_to_tx_features.drop_duplicates(subset=['input_address', 'txId', "Time step"])
wallets_to_tx_features = wallets_to_tx_features.rename(columns={"input_address": "addrId"})
wallets_to_tx_features.drop("txId", axis=1, inplace=True)
wallets_to_tx_features["whole_fee_5"] = (np.floor(wallets_to_tx_features["fees"] * (10**5)) == (wallets_to_tx_features["fees"] * (10**5))).astype(int)
wallets_to_tx_features.drop("fees", axis=1, inplace=True)
wallets_to_tx_features


,addrId,Time step,whole_fee_5
0,14YRXHHof4BY1TVxN5FqYPcEdpmXiYT78a,1,1
1,13Lhad3SAmu2vqYg2dxbNcxH7LE77kJu2w,1,1
2,1MAQQZn7EHP6J3erXByCciFiVcgS8ZhWqz,1,1
3,16zs5SVSyADh5WrLNbZbpRLsBsN5uEzgeK,1,1
4,1QJpwtUorBKPGUJkSyrRcBKTAHq4CXrdYh,1,1
...,...,...,...
477112,1HdnGvuc21Y4QfBEHUc3NFRJhGywdSFUb,49,1
477113,3MfN5to5K5be2RupWE8rjJHQ6V9L8ypWeh,49,0
477114,3DzbpEogZ1mn9FgCHcmzYPLDbV9GuxYHpi,49,0
477115,34yD1sQg6C16aANCtibYXRj5NsX6tt4v5R,49,0


In [107]:
addr_to_whole_in_ts = wallets_to_tx_features.groupby(["addrId", "Time step"])["whole_fee_5"].sum().reset_index()
addr_to_whole_in_ts = addr_to_whole_in_ts.rename(columns={"whole_fee_5": "whole_fees_in_ts_5"})
addr_to_whole_in_ts

,addrId,Time step,whole_fees_in_ts_5
0,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,17,0
1,1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ,5,1
2,111HRAJxnoxqyKRVnjqBmwqneUrHc1chi,23,0
3,111Ud9zxFi3VgcstEM2R3YAS3CNxf9o8r,33,0
4,111Y3BkUGLSWQjHCb2Mg8oMNiwc1jxMwe,36,0
...,...,...,...
417520,3R2KK3y2JcFvczaeZ43mhqj8898DPKG3rD,36,0
417521,3R2LuyL8ktL9kDAXLDisrXZvdJteAMWMAp,24,1
417522,3R2PkovCpT6ZkV6ubHUR5u8sQJqmKKKAxA,35,0
417523,3R2Uw5MRdSSigp8AjfT7K5es6Hupm4qLSq,31,0


In [108]:
addr_to_whole_fee = wallets_to_tx_features.groupby(["addrId"])["whole_fee_5"].sum().reset_index()
addr_to_whole_fee = addr_to_whole_fee.rename(columns={"whole_fee_5": "whole_fees_5"})
addr_to_whole_fee

,addrId,whole_fees_5
0,111218KKkh1JJFRHbwM16AwCiVCc4m7he1,0
1,1117wASFaYgJJP6MiY8cPD5DMdQda8gDZ,1
2,111HRAJxnoxqyKRVnjqBmwqneUrHc1chi,0
3,111Ud9zxFi3VgcstEM2R3YAS3CNxf9o8r,0
4,111Y3BkUGLSWQjHCb2Mg8oMNiwc1jxMwe,0
...,...,...
400207,3R2KK3y2JcFvczaeZ43mhqj8898DPKG3rD,0
400208,3R2LuyL8ktL9kDAXLDisrXZvdJteAMWMAp,1
400209,3R2PkovCpT6ZkV6ubHUR5u8sQJqmKKKAxA,0
400210,3R2Uw5MRdSSigp8AjfT7K5es6Hupm4qLSq,0


In [109]:
wallets_to_fees_median = pd.merge(
    left=wallets_to_tx,
    right=tx_to_fee,
    left_on="txId",
    right_on="txId",
    how="left",
)
wallets_to_fees_median = wallets_to_fees_median.drop_duplicates(subset=['input_address', 'txId'])
wallets_to_fees_median = wallets_to_fees_median.rename(columns={"input_address": "addrId"})
wallets_to_fees_median.drop("txId", axis=1, inplace=True)
wallets_to_fees_median.drop("Time step", axis=1, inplace=True)

wallets_to_fees_median = wallets_to_fees_median.groupby(["addrId"])["fees"].median().reset_index()
wallets_to_fees_median = wallets_to_fees_median.rename(columns={"fees": "fees_median"})
wallets_to_fees_median.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400212 entries, 0 to 400211
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   addrId       400212 non-null  object 
 1   fees_median  400212 non-null  float64
dtypes: float64(1), object(1)
memory usage: 6.1+ MB


In [110]:
wallets_with_users_features["addrId"].nunique()

822942

In [111]:
wallets_to_whole_features = pd.merge(
    left=wallets_with_users_features,
    right=wallet_to_gini,
    left_on="addrId",
    right_on="addrId",
    how="left",
)
wallets_to_whole_features.drop("fees_median", axis=1, inplace=True)
wallets_to_whole_features = pd.merge(
    left=wallets_to_whole_features,
    right=addr_to_whole_fee,
    left_on="addrId",
    right_on="addrId",
    how="left",
)
wallets_to_whole_features = pd.merge(
    left=wallets_to_whole_features,
    right=wallets_to_fees_median,
    left_on="addrId",
    right_on="addrId",
    how="left",
)

wallets_to_whole_features = pd.merge(
    left=wallets_to_whole_features,
    right=addr_to_whole_in_ts,
    on=["addrId", "Time step"],
    how="left",
)
wallets_to_whole_features["whole_fees_5"] = wallets_to_whole_features["whole_fees_5"].fillna(0)
wallets_to_whole_features["whole_fees_in_ts_5"] = wallets_to_whole_features["whole_fees_in_ts_5"].fillna(0)
wallets_to_whole_features

,Time step,class,num_txs_as_sender,num_txs_as receiver,first_block_appeared_in,last_block_appeared_in,lifetime_in_blocks,total_txs,first_sent_block,first_received_block,...,user_btc_sent_total,user_btc_received_total,user_interracted_output_address_cnt,user_interracted_input_address_cnt,user_overall_activity_coef,user_whole_fee_5,addr_gini,whole_fees_5,fees_median,whole_fees_in_ts_5
0,25,2,0.0,1.0,439586.0,439586.0,0.0,1.0,0.0,439586.0,...,0.000000,0.010628,0.0,24.0,0.000000,0.0,0.00000,0.0,NaN,0.0
1,25,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.000000,2.184366,0.0,8.0,0.106629,0.0,0.12482,0.0,NaN,0.0
2,29,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.000000,2.184366,0.0,8.0,0.106629,0.0,0.12482,0.0,NaN,0.0
3,39,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.000000,2.184366,0.0,8.0,0.106629,0.0,0.12482,0.0,NaN,0.0
4,39,3,0.0,8.0,439589.0,485959.0,46370.0,8.0,0.0,439589.0,...,0.000000,2.184366,0.0,8.0,0.106629,0.0,0.12482,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268255,31,3,1.0,1.0,451684.0,451685.0,1.0,2.0,451685.0,451684.0,...,1.585620,1.585620,2.0,1.0,0.000000,0.0,0.00000,0.0,0.000320,0.0
1268256,44,3,0.0,1.0,477895.0,477895.0,0.0,1.0,0.0,477895.0,...,0.000000,0.063876,0.0,1.0,0.000000,0.0,0.00000,0.0,NaN,0.0
1268257,9,3,0.0,1.0,407342.0,407342.0,0.0,1.0,0.0,407342.0,...,0.000000,0.003801,0.0,1.0,0.000000,0.0,0.00000,0.0,NaN,0.0
1268258,3,3,0.0,1.0,395235.0,395235.0,0.0,1.0,0.0,395235.0,...,0.000000,0.020718,0.0,1.0,0.000000,0.0,0.00000,0.0,NaN,0.0


In [113]:
wallets_to_whole_features.to_csv("../dataset/custom/wallets_features_extended.csv", index=False)

# Агрегация до wallet_id (против wallet_id + Time step)

In [114]:
wallets_to_whole_features.columns

Index(['Time step', 'class', 'num_txs_as_sender', 'num_txs_as receiver',
       'first_block_appeared_in', 'last_block_appeared_in',
       'lifetime_in_blocks', 'total_txs', 'first_sent_block',
       'first_received_block', 'num_timesteps_appeared_in',
       'btc_transacted_total', 'btc_transacted_min', 'btc_transacted_max',
       'btc_transacted_mean', 'btc_transacted_median', 'btc_sent_total',
       'btc_sent_min', 'btc_sent_max', 'btc_sent_mean', 'btc_sent_median',
       'btc_received_total', 'btc_received_min', 'btc_received_max',
       'btc_received_mean', 'btc_received_median', 'fees_total', 'fees_min',
       'fees_max', 'fees_mean', 'fees_as_share_total', 'fees_as_share_min',
       'fees_as_share_max', 'fees_as_share_mean', 'fees_as_share_median',
       'blocks_btwn_txs_total', 'blocks_btwn_txs_min', 'blocks_btwn_txs_max',
       'blocks_btwn_txs_mean', 'blocks_btwn_txs_median',
       'blocks_btwn_input_txs_total', 'blocks_btwn_input_txs_min',
       'blocks_btwn_inpu

In [115]:
GROUPBY_FIELDS = [
    'class',
     
    'num_timesteps_appeared_in',
    "fees_median",
    # 'btc_transacted_mean', 'btc_transacted_median', 
    #  'btc_sent_mean', 'btc_sent_median',
    
    # 'btc_received_mean', 'btc_received_median',
    #  'fees_mean', 'fees_median', 'fees_as_share_total',

    # 'fees_as_share_min', 'fees_as_share_max', 'fees_as_share_mean',
    # 'fees_as_share_median',
    
    # 'blocks_btwn_txs_total', 'blocks_btwn_txs_min',
    # 'blocks_btwn_txs_max', 'blocks_btwn_txs_mean', 'blocks_btwn_txs_median',
    # 'blocks_btwn_input_txs_total', 'blocks_btwn_input_txs_min',
    # 'blocks_btwn_input_txs_max', 'blocks_btwn_input_txs_mean',
    # 'blocks_btwn_input_txs_median', 'blocks_btwn_output_txs_total',
    # 'blocks_btwn_output_txs_min', 'blocks_btwn_output_txs_max',
    # 'blocks_btwn_output_txs_mean', 'blocks_btwn_output_txs_median',

    'num_addr_transacted_multiple', 
    # 'transacted_w_address_mean', 'transacted_w_address_median',
    
    'addrId',

    'userId', 'user_ts_fees_share_mean', 'user_ts_fees_share_min',
    'user_ts_fees_share_max', 'user_addr_cnt', 'user_outcoming_tx_cnt',
    'user_incoming_tx_cnt', 'user_input_users_cnt', 'user_output_users_cnt',
    'user_active_time_steps_cnt', 'user_btc_sent_total',
    'user_btc_received_total', 'user_interracted_output_address_cnt',
    'user_interracted_input_address_cnt', 'user_overall_activity_coef',
    'user_whole_fee_5',
    'addr_gini',

    'whole_fees_5',
    # 'whole_fees_in_ts_5'
]

AGG_FIELDS_SUM = [
    'num_txs_as_sender', 'num_txs_as receiver',
    'lifetime_in_blocks', 'total_txs',
     'btc_transacted_total',  'btc_sent_total', 'btc_received_total',
      'fees_total', 'transacted_w_address_total',
    
]
AGG_FIELDS_MIN = [
    'first_block_appeared_in', 'first_sent_block', 'first_received_block',
    'btc_transacted_min', 
    'btc_sent_min', 'btc_received_min', 
     'fees_min', 'transacted_w_address_min',
]
AGG_FIELDS_MAX = [
    'btc_transacted_max', 'btc_sent_max', 'btc_received_max', 'fees_max',
    'last_block_appeared_in',  'transacted_w_address_max',
]
AGG_FIELDS_MEDIAN = [
    'fees_median'
]

In [121]:
wallets_to_whole_features["fees_median"] = wallets_to_whole_features["fees_median"].fillna(0)

In [122]:
wallets_features_agg = wallets_to_whole_features \
    .groupby(GROUPBY_FIELDS) \
    .agg({
        **{key: "sum" for key in AGG_FIELDS_SUM},
        **{key: "min" for key in AGG_FIELDS_MIN},
        **{key: "max" for key in AGG_FIELDS_MAX},
        
    }) \
    .reset_index()


In [123]:
wallets_features_agg["addrId"].nunique()

822942

In [124]:
wallets_features_agg.to_csv("../dataset/custom/wallets_features_aggregated.csv", index=False)

In [ ]:
wallets_features